# Variable Calculation

This Jupyter Notebook enables the calculation of additional variables from the given dataset which can be further used in the model selection & model training.

The input needed for this script is a dataset containing weather variables for one or more locations. The first column of the dataframe should contain the day of observation in the format of YYYY-MM-DD. Further, the dataframe should contain a column called 'city' that contains the name of the weather station. The following naming convention is applied for weather variables here:

- ws: windspeed
- t: temperature
- wd: wind direction in degrees
- h: humidity relative to temperature
- p: precipitation
- sun: solar irradiance
- press: air pressure
- dew: dew point

Not all of these variables have to be supplied. 


## Load Libraries & Data

In [156]:
# import libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import math
from sklearn.decomposition import PCA, KernelPCA

In [157]:
# input path to raw csv file with air pollution and weather data
data_path =  './che/df.csv'

In [158]:
# read in dataset
df = pd.read_csv(data_path, encoding='latin1', index_col =0)

In [159]:
# get an overview of the variables 
print('The dataset contains the following variables:', df.columns.values)

The dataset contains the following variables: ['pm10' 'co' 'press' 'h' 't' 'o3' 'p' 'nox' 'no2' 'no' 'ws' 'wd' 'city']


In [160]:
df['time'] = df.index

In [161]:
# set index to datetime format value of time column & drop time column
df.index = pd.to_datetime(df.index, format ='%d.%m.%Y %H:%M')

In [162]:
df

,pm10,co,press,h,t,o3,p,nox,no2,no,ws,wd,city,time
time,,,,,,,,,,,,,,
2016-01-01 01:00:00,31.51,NaN,974.54,89.33,5.12,-0.75,0.00,60.02,33.86,52.77,0.41,335.77,Opfikon_Balsberg,01.01.2016 01:00
2016-01-01 02:00:00,51.92,NaN,974.54,89.35,5.20,-0.78,0.00,71.88,41.80,62.39,0.56,291.10,Opfikon_Balsberg,01.01.2016 02:00
2016-01-01 03:00:00,57.19,NaN,974.74,89.04,5.39,-0.72,40.15,62.91,35.33,55.43,0.22,36.05,Opfikon_Balsberg,01.01.2016 03:00
2016-01-01 04:00:00,54.42,NaN,975.12,89.66,5.50,-0.74,0.00,72.25,35.36,67.06,0.21,90.56,Opfikon_Balsberg,01.01.2016 04:00
2016-01-01 05:00:00,59.30,NaN,975.59,89.73,5.42,-0.79,0.00,62.41,25.19,61.41,0.86,81.19,Opfikon_Balsberg,01.01.2016 05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-06 20:00:00,27.04,0.40,949.19,85.72,7.42,2.08,34.14,34.07,20.36,1.45,10.47,NaN,Zuerich_Stampfenbachstrasse,06.02.2021 20:00
2021-02-06 21:00:00,26.88,0.35,948.71,87.63,7.09,1.99,27.35,31.19,13.77,1.49,2.00,NaN,Zuerich_Stampfenbachstrasse,06.02.2021 21:00
2021-02-06 22:00:00,27.17,0.39,949.08,88.07,7.05,2.42,30.78,32.65,17.09,0.83,314.67,NaN,Zuerich_Stampfenbachstrasse,06.02.2021 22:00


In [163]:
# get vector with names of all the cities
cities = np.unique(df['city'].values)
print('The dataset covers the following cities:', cities)

The dataset covers the following cities: ['Opfikon_Balsberg' 'StGallen_Blumenbergplatz' 'StGallen_Stuelegg'
 'Zuerich_Schimmelstrasse' 'Zuerich_Stampfenbachstrasse']


In [164]:
# aggregate per day
df = df.groupby([df.index.date, df['city']]).mean()

In [165]:
# set datetime index again
df['city'] = df.index.to_frame()['city']
df = df.droplevel('city')


In [166]:
df.index = pd.to_datetime(df.index, format = '%Y-%m-%d')

In [167]:
df['time'] = df.index

### Day of the Year
This variable refers to the day of the given year, i.e. 1st of January = 1.

In [168]:
df['day_year'] = df.time.dt.dayofyear

In [169]:
df =df.drop('time',axis =1)

In [170]:
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=['p','h', 't'])

In [171]:
df

,pm10,co,press,h,t,o3,p,nox,no2,no,ws,wd,city,day_year
2016-01-01,33.401818,NaN,973.206818,87.395909,5.157727,14.421364,1.825000,49.456818,32.972727,40.180000,0.796522,99.970435,Opfikon_Balsberg,1
2016-01-01,22.060000,0.372727,943.960000,101.996522,4.385217,19.469000,1.164348,23.590000,29.195000,10.382000,0.731739,123.624783,StGallen_Blumenbergplatz,1
2016-01-01,8.542174,NaN,918.646087,97.744783,2.931304,42.839565,2.068261,6.845652,9.458696,2.370000,1.009130,151.439565,StGallen_Stuelegg,1
2016-01-01,68.412174,NaN,NaN,87.422174,6.094783,19.359565,1.064348,44.335652,37.208696,31.030435,0.684348,134.181739,Zuerich_Schimmelstrasse,1
2016-01-01,57.794783,0.420435,973.026522,89.577391,5.638696,17.854783,2.638261,37.371739,32.528696,25.395217,1.354783,86.635652,Zuerich_Stampfenbachstrasse,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-07,21.660000,NaN,950.560000,89.650000,5.000000,0.840000,0.000000,26.340000,18.330000,20.900000,1.300000,86.160000,Opfikon_Balsberg,38
2021-02-07,23.860000,NaN,922.060000,91.550000,5.250000,3.560000,0.000000,31.190000,35.960000,15.450000,0.780000,223.570000,StGallen_Blumenbergplatz,38
2021-02-07,73.080000,NaN,896.840000,49.570000,9.660000,43.440000,0.000000,5.680000,11.690000,-0.540000,0.810000,263.380000,StGallen_Stuelegg,38
2021-02-07,31.450000,NaN,NaN,82.160000,7.590000,2.380000,0.000000,44.140000,33.650000,33.110000,0.170000,11.210000,Zuerich_Schimmelstrasse,38


## Calculate Additional Variables

### Dew Point
The dew point indicates the amount of moisture in the air. The higher the dew point, the higher the moisture content of the air at a given temperature. If not included in the dataset, the dew point can approximated using the following simple formula:

$$
t_{dp} = t - \frac{100- h}{5}
$$

where $t_{dp}$ is the dew point, $t$ is the temperature and $h$ is relative humidity.


In [172]:
if not('dew' in df.columns):
    print('calculated dew')
    df['dew'] = df['t'] -((100-df['h'])/5)

calculated dew


## Principal Component Analysis
Prinicipal Component Analysis (PCA) is a technique to reduce dimensionality. As humidity, temperature, dew point and precipation are often highly correlated (due to the definition of these variables or due to environmental relationships), PCA offers a technique to create a variable that captures most of the variablity present in these variables. This can help reduce collinearity in the final model (which in turn can lead to incorrect attribution of effects to variables). The first component calculated through PCA is added to the dataset with the name 'pca'.

For more information on collinearity: https://doi.org/10.1111/j.1600-0587.2012.07348.x

In [173]:
# perform principal component analysis for humidity, dew point, temperature and precipitation (if all of these are available)

# initalize PCA model & fit to dataset
n = 0
pca_vars = ['p','h', 'dew','t']
for var in pca_vars:
    if var in df.columns:
        n = n + 1
    else:
        pca_vars.remove(var)
print(pca_vars)
pca=PCA(n_components = n) 
_pca = pca.fit(df[pca_vars]) 

# get one transformed variable
transformed = pca.fit_transform(df[pca_vars])[:,0]
df['pca'] = transformed

['p', 'h', 'dew', 't']


Calculation of various long-term lag variables for the first principal component variable.

In [174]:
df['lagpca_year'] =df.pca.rolling(window=336, min_periods =1).mean()
df['lagpca_halfyear'] =df.pca.rolling(window=168, min_periods =1).mean()
df['lagpca_12weeks'] = df.pca.rolling(window=84, min_periods =1).mean()
df['lagpca_8weeks'] = df.pca.rolling(window=56, min_periods =1).mean()
df['lagpca_4weeks'] = df.pca.rolling(window=28, min_periods =1).mean()
df['lagpca_2weeks'] = df.pca.rolling(window=14, min_periods =1).mean()
df['lagpca_1week'] = df.pca.rolling(window=7, min_periods =1).mean()

## Long-Term Windspeed Lags
As wind speed is known to have a big influence on air pollution in general and often in many cities the current air pollution levels are also dependent on the wind speeds the weeks before, laggeed variables with the mean and maximum windspeed through the last 1 week, 2 weeks and 4weeks are calculated. 

In [175]:
df['lagws_4weeks'] = df.ws.rolling(window=28, min_periods =1).mean()
df['lagws_2weeks'] = df.ws.rolling(window=14, min_periods =1).mean()
df['lagws_1week'] = df.ws.rolling(window=7, min_periods=1).mean()

df['lagws_4weeks_max'] = df.ws.rolling(window=28, min_periods =1).max()
df['lagws_2weeks_max'] = df.ws.rolling(window=14, min_periods =1).max()
df['lagws_1week_max'] = df.ws.rolling(window=7, min_periods=1).max()

## Year, Month & Weekday

Additional categorical variables are added for the year, month and weekday of an observation.

In [176]:
# calculate additional time variables
df['year'] = df.index.year
df['weekday'] = df.index.weekday
df['month'] = df.index.month

## Winddirection Transformation
The wind direction variable is ransformed from a linear scale of wind direction with polar coordinates (0-360°) to Cartesian coordinates, where angles increase clockwise. This allows for a more accurate feature representation of the data for wind direction around the north axis. We call the resulting variables representing x- and y- coordinates of the wind direction 'wx' and 'wy'.

In [177]:
df['wx'] = np.sin(df['wd']/360 * 2*math.pi)
df['wy'] = np.cos(df['wd']/360 * 2*math.pi) 

Normalize humidity by temperature

In [178]:
df['h'] = df['h']/df['t']

## Lagged Variables
Often not only the current weather situation influences air pollution but also the weather of the previous days. Therefore, 1-day, 2-days and 3-days lags are caluclated for all weather variables.

In [179]:
# shfit values one period
df['h_lag1'] = df['h'].shift()
df['t_lag1'] = df['t'].shift()
df['wx_lag1'] = df['wx'].shift()
df['wy_lag1'] = df['wy'].shift()
df['ws_lag1'] = df['ws'].shift()
df['dew_lag1'] = df['dew'].shift()

In [180]:
# shift values two periods
df['h_lag2'] = df['h'].shift(periods =2)
df['t_lag2'] = df['t'].shift(periods =2)
df['wx_lag2'] = df['wx'].shift(periods =2)
df['wy_lag2'] = df['wy'].shift(periods =2)
df['ws_lag2'] = df['ws'].shift(periods =2)
df['dew_lag2'] = df['dew'].shift(periods =2)

In [181]:
# shift values three periods
df['h_lag3'] = df['h'].shift(periods =3)
df['t_lag3'] = df['t'].shift(periods =3)
df['wx_lag3'] = df['wx'].shift(periods =3)
df['wy_lag3'] = df['wy'].shift(periods =3)
df['ws_lag3'] = df['ws'].shift(periods =3)
df['dew_lag3'] = df['dew'].shift(periods = 3)

This dataframe with additional variables can be used to perform the model selection algorithm. Details are given in the script "Model Selection".

In [182]:
df

,pm10,co,press,h,t,o3,p,nox,no2,no,...,wx_lag2,wy_lag2,ws_lag2,dew_lag2,h_lag3,t_lag3,wx_lag3,wy_lag3,ws_lag3,dew_lag3
2016-01-01,33.401818,NaN,973.206818,16.944655,5.157727,14.421364,1.825000,49.456818,32.972727,40.180000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01,22.060000,0.372727,943.960000,23.259171,4.385217,19.469000,1.164348,23.590000,29.195000,10.382000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01,8.542174,NaN,918.646087,33.345150,2.931304,42.839565,2.068261,6.845652,9.458696,2.370000,...,0.984897,-0.173140,0.796522,2.636909,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01,68.412174,NaN,NaN,14.343772,6.094783,19.359565,1.064348,44.335652,37.208696,31.030435,...,0.832682,-0.553752,0.731739,4.784522,16.944655,5.157727,0.984897,-0.173140,0.796522,2.636909
2016-01-01,57.794783,0.420435,973.026522,15.886190,5.638696,17.854783,2.638261,37.371739,32.528696,25.395217,...,0.478085,-0.878313,1.009130,2.480261,23.259171,4.385217,0.832682,-0.553752,0.731739,4.784522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-07,21.660000,NaN,950.560000,17.930000,5.000000,0.840000,0.000000,26.340000,18.330000,20.900000,...,-0.859027,-0.511930,0.355000,4.033500,7.156596,8.440000,0.510524,-0.859863,1.102500,0.520333
2021-02-07,23.860000,NaN,922.060000,17.438095,5.250000,3.560000,0.000000,31.190000,35.960000,15.450000,...,NaN,NaN,137.134583,4.416500,10.756228,7.626667,-0.859027,-0.511930,0.355000,4.033500
2021-02-07,73.080000,NaN,896.840000,5.131470,9.660000,43.440000,0.000000,5.680000,11.690000,-0.540000,...,0.997755,0.066970,1.300000,2.930000,12.339212,7.040833,NaN,NaN,137.134583,4.416500
2021-02-07,31.450000,NaN,NaN,10.824769,7.590000,2.380000,0.000000,44.140000,33.650000,33.110000,...,-0.689240,-0.724533,0.780000,3.560000,17.930000,5.000000,0.997755,0.066970,1.300000,2.930000


In [155]:
df.to_csv('.che/df2.csv')